# Importing Pandas and Display.

In [ ]:

import pandas as pd
from IPython.display import display 

# The next 3 blocks run the CSV file and then sorts it the same way.

In [ ]:
df_code = pd.read_csv('Module6_claims copy/clean/example_2/STONYBRK_20240531_CODE.csv')

# 1. Shape (rows, columns)
print("Shape:", df_code.shape)

# 2. First 5 rows
print("\nFirst 5 rows:")
display(df_code.head())

# 3. Column names and data types
print("\nColumn names and data types:")
print(df_code.dtypes)

# 4. Missing value counts per column
print("\nMissing values per column:")
print(df_code.isnull().sum())

# 5. Basic descriptive statistics (numeric columns only)
print("\nDescriptive statistics for numeric columns:")
display(df_code.describe())


In [ ]:
df_header =pd.read_csv('Module6_claims copy/clean/example_2/STONYBRK_20240531_HEADER.csv')

# 1. Shape (rows, columns)
print("Shape:", df_header.shape)

# 2. First 5 rows
print("\nFirst 5 rows:")
display(df_header.head())

# 3. Column names and data types
print("\nColumn names and data types:")
print(df_header.dtypes)

# 4. Missing value counts per column
print("\nMissing values per column:")
print(df_header.isnull().sum())

# 5. Basic descriptive statistics (numeric columns only)
print("\nDescriptive statistics for numeric columns:")
display(df_header.describe())

In [ ]:
df_line = pd.read_csv('Module6_claims copy/clean/example_2/STONYBRK_20240531_LINE.csv')

# 1. Shape (rows, columns)
print("Shape:", df_line.shape)

# 2. First 5 rows
print("\nFirst 5 rows:")
display(df_line.head())

# 3. Column names and data types
print("\nColumn names and data types:")
print(df_line.dtypes)

# 4. Missing value counts per column
print("\nMissing values per column:")
print(df_line.isnull().sum())

# 5. Basic descriptive statistics (numeric columns only)
print("\nDescriptive statistics for numeric columns:")
display(df_line.describe())

# This just let me see what the column headings were. Need to run to run the rest of the code

In [ ]:
import pandas as pd

paths = ["Module6_claims copy/clean/example_2/STONYBRK_20240531_CODE.csv", "Module6_claims copy/clean/example_2/STONYBRK_20240531_HEADER.csv", "Module6_claims copy/clean/example_2/STONYBRK_20240531_LINE.csv"]
dfs = [pd.read_csv(p) for p in paths]

for i, df in enumerate(dfs, start=1):
    print(f"\n=== Dataset {i} ===")
    print(df.shape)
    print(df.columns)

#  Billing Providers by Name
## This code block takes information from Dataset 2 and takes information from the columns "BillingProvFirstName" and "BillingProvLastName" to combine them into the new column "BillingProvName". Then it groups it by BillingProvName and BillingProvNPI to sort it by claim counts. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df2 = dfs[1]   # dataset 2

# Create a full provider name column
df2["BillingProviderName"] = (
    df2["BillingProvFirstName"].fillna("") + " " +
    df2["BillingProvLastName"].fillna("")
).str.strip()

# Group by Billing Provider Name + NPI
top5 = (
    df2.groupby(["BillingProviderName", "BillingProviderNPI"])
       .size()
       .reset_index(name="claim_count")
       .sort_values("claim_count", ascending=False)
       .head(5)
)

print(top5)

# Bar Chart of Top 5 Billing Providers

In [ ]:
plt.figure(figsize=(10,5))

labels = (
    top5["BillingProviderName"] 
    + "\nNPI: " 
    + top5["BillingProviderNPI"].astype(str)
)

plt.bar(labels, top5["claim_count"], color="skyblue")

plt.xticks(rotation=45, ha="right")
plt.ylabel("Number of Claims")
plt.title("Top 5 Billing Providers by Claim Count")
plt.tight_layout()
plt.show()

#  Top 5 Primary Payers by Claim Volume
## This uses df2 = dfs[1] or df2 = Dataframe 2. It fiters through the headers until it reachers "PrimaryPayerName" and then sorts it my number of claims from greatest to least. Then it outputs the top 5.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df2 = dfs[1]   

# Count claims by primary payer
payer_counts = (
    df2["PrimaryPayerName"]
    .value_counts()
    .reset_index()
)

payer_counts.columns = ["PrimaryPayerName", "claim_count"]


top5_payers = payer_counts.head(5)
print(top5_payers)

# Percentage of Total Claim

## Takes the claim count calcualted previously and divides it by the total number of claims and multiplties that number by 100. Then it rounds to the second decimal place. 

In [ ]:
total_claims = payer_counts["claim_count"].sum()

top5_payers["percentage"] = (
    top5_payers["claim_count"] / total_claims * 100
).round(2)

print(top5_payers)


# Pie Chart showing Payer Distribution

In [ ]:
plt.figure(figsize=(8,8))
plt.pie(
    top5_payers["claim_count"],
    labels=top5_payers["PrimaryPayerName"],
    autopct="%1.1f%%",
    startangle=90
)

plt.title("Primary Payer Distribution (Top 5)")
plt.show()

# Most Frequent Code Distributions


In [ ]:
import pandas as pd

df1 = dfs[0]   # Dataset 1

# Count diagnosis codes
dx_counts = (
    df1["CodeValue"]
    .value_counts()
    .reset_index()
)

dx_counts.columns = ["ICD10_Code", "count"]

# Top 10
top10_dx = dx_counts.head(10)
print(top10_dx)

# Bar Chart of Top 10 diagnosis code

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))

plt.bar(top10_dx["ICD10_Code"], top10_dx["count"], color="purple")

plt.xticks(rotation=45, ha="right")
plt.xlabel("ICD-10 Diagnosis Code")
plt.ylabel("Frequency")
plt.title("Top 10 Most Frequent Diagnosis Codes")
plt.tight_layout()
plt.show()

# Most Frequently Billed HCPCS

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df3 = dfs[2]   # dataset 3

# Pick your preferred description field
desc_col = "CodeName"    # or "ClientProcedureName" if more accurate

# Keep only rows that actually have an HCPCS code
df_valid = df3[df3["HCPCS"].notna() & (df3["HCPCS"] != "")]

# Count HCPCS codes + descriptions
hcpcs_counts = (
    df_valid.groupby(["HCPCS", desc_col])
            .size()
            .reset_index(name="frequency")
            .sort_values("frequency", ascending=False)
)

top10_hcpcs = hcpcs_counts.head(10)
print(top10_hcpcs)

# Bar Chart of HCPCS

In [ ]:
plt.figure(figsize=(12,6))

plt.bar(
    top10_hcpcs["HCPCS"].astype(str),
    top10_hcpcs["frequency"],
    color="teal"
)

plt.xticks(rotation=45, ha="right")
plt.xlabel("HCPCS Procedure Code")
plt.ylabel("Frequency")
plt.title("Top 10 Most Frequently Billed HCPCS Codes")
plt.tight_layout()
plt.show()

# Count of Claim by Place of Service
## Takes PlaceOfService from dataframe 2 and counts each claim for each POS. Then it turns the POS numbers into a readable location. After, it counts them and then breaks them up based on percentage.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df2 = dfs[1]

print(df2["PlaceOfService"].head())     

pos_counts = (
    df2["PlaceOfService"]
      .value_counts(dropna=False)
      .reset_index()
)

pos_counts.columns = ["PlaceOfServiceCode", "claim_count"]
print(pos_counts)

# Map POS codes to human-readable descriptions
pos_desc_map = {
    11: "DOCTOR'S OFFICE",
    21: "INPATIENT HOSPITAL",
}

# Make sure codes are integers where possible
df2["POS_Code"] = df2["PlaceOfService"].astype("Int64")   
df2["POS_Desc"] = df2["POS_Code"].map(pos_desc_map)


pos_counts_labeled = (
    df2.groupby(["POS_Code", "POS_Desc"])
       .size()
       .reset_index(name="claim_count")
       .sort_values("claim_count", ascending=False)
)

print(pos_counts_labeled)


total_claims = len(df2)

inpatient_count = df2[df2["POS_Code"] == 21].shape[0]
office_count    = df2[df2["POS_Code"] == 11].shape[0]

inpatient_pct = round(inpatient_count / total_claims * 100, 2)
office_pct    = round(office_count    / total_claims * 100, 2)

print(f"Total claims: {total_claims}")
print(f"INPATIENT (21): {inpatient_count} claims ({inpatient_pct}%)")
print(f"DOCTOR'S OFFICE (11): {office_count} claims ({office_pct}%)")

# Bar chart for POS Distribution

In [ ]:
plt.figure(figsize=(10,5))
plt.figure(figsize=(6,6))
plt.pie(
    [inpatient_count, office_count],
    labels=["INPATIENT", "DOCTOR'S OFFICE"],
    autopct="%1.1f%%",
    startangle=90,
    colors=["steelblue", "lightgreen"]
)

plt.title("INPATIENT vs DOCTOR'S OFFICE Claim Distribution")
plt.show()


# Question 6: Claims with High Service Line Counts

In [ ]:
merged = pd.merge(
    df_header,
    df_line,
    on="ProspectiveClaimId",
    how="left"
)


line_counts = (
    merged.groupby("ProspectiveClaimId")
          .size()
          .reset_index(name="num_lines")
)


claims_5plus = line_counts[line_counts["num_lines"] >= 5]


df_header["ProviderName"] = (
    df_header["BillingProvFirstName"].fillna("") + " " +
    df_header["BillingProvLastName"].fillna("")
).str.strip()


charges_per_claim = (
    df_line.groupby("ProspectiveClaimId")["Charges"]
           .sum()
           .reset_index(name="total_charges")
)


result = (
    claims_5plus
    .merge(df_header[["ProspectiveClaimId", "ProviderName"]], on="ProspectiveClaimId", how="left")
    .merge(charges_per_claim, on="ProspectiveClaimId", how="left")
)


result = result.sort_values("num_lines", ascending=False)

print(result[["ProspectiveClaimId", "ProviderName", "num_lines", "total_charges"]])

# Question 7: Diagnosis-Procedure Combinations

In [ ]:
#assigning shorter names for dataset

df_dx = dfs[0]   
df_hd = dfs[1]   
df_ln = dfs[2]  

# Mergeing HEADER + LINE first
merged_hl = pd.merge(
    df_hd,
    df_ln,
    on="ProspectiveClaimId",
    how="left"
)

# Then mergeing with DIAGNOSIS file
merged_full = pd.merge(
    merged_hl,
    df_dx[["ProspectiveClaimId", "CodeValue"]],
    on="ProspectiveClaimId",
    how="left"
)

cpt_99291 = merged_full[merged_full["HCPCS"] == "99291"]


dx_counts_99291 = (
    cpt_99291["CodeValue"]
    .value_counts()
    .reset_index()
)

dx_counts_99291.columns = ["ICD10_Code", "count"]
print(dx_counts_99291.head(10))


most_common_dx = dx_counts_99291.iloc[0]
print("Most common diagnosis for CPT 99291:")
print(most_common_dx)

# Question 8: Charges by Payer

In [ ]:
df_header = df2   
df_line   = df3 

merged = pd.merge(
    df_header,
    df_line[["ProspectiveClaimId", "Charges"]],  
    on="ProspectiveClaimId",
    how="left"
)

charges_per_claim = (
    merged.groupby("ProspectiveClaimId")["Charges"]
          .sum()
          .reset_index(name="TotalCharges")
)

payer_info = df_header[["ProspectiveClaimId", "PrimaryPayerName"]]

charges_with_payer = pd.merge(
    charges_per_claim,
    payer_info,
    on="ProspectiveClaimId",
    how="left"
)

payer_stats = (
    charges_with_payer.groupby("PrimaryPayerName")
        .agg(
            TotalCharges = ("TotalCharges", "sum"),
            AvgCharges   = ("TotalCharges", "mean"),
            NumClaims    = ("ProspectiveClaimId", "count")
        )
        .reset_index()
)

top10_payers = payer_stats.sort_values(
    "TotalCharges",
    ascending=False
).head(10)

print(top10_payers)

# Question 9: Your Own Analysis

In [ ]:
df_header = df2     
df_line   = df3     

merged = pd.merge(
    df_header[["ProspectiveClaimId", "PlaceOfService"]],
    df_line[["ProspectiveClaimId", "Charges"]],
    on="ProspectiveClaimId",
    how="left"
)

charges_per_claim = (
    merged.groupby(["ProspectiveClaimId", "PlaceOfService"])["Charges"]
          .sum()
          .reset_index()
)

pos_summary = (
    charges_per_claim.groupby("PlaceOfService")["Charges"]
                     .agg(["count", "mean", "sum"])
                     .reset_index()
                     .rename(columns={
                         "count": "NumClaims",
                         "mean": "AvgCharges",
                         "sum": "TotalCharges"
                     })
)

print(pos_summary.sort_values("AvgCharges", ascending=False))

# Visualization of Question 9

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
plt.bar(pos_summary["PlaceOfService"].astype(str),
        pos_summary["AvgCharges"],
        color="steelblue")

plt.xticks(rotation=45, ha="right")
plt.xlabel("Place of Service")
plt.ylabel("Average Charges ($)")
plt.title("Average Charges by Place of Service")
plt.tight_layout()
plt.show()